In [1]:
import os 

In [2]:
%pwd

'/Users/melihaltin/Documents/Development/data-science/Lung-Cancer/Lung-Cancer/research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir : Path
    data_source : str
    local_data_file : Path
    unzip_dir : Path

In [5]:
from Lung_Cancer.constants import *
from Lung_Cancer.utils.common import read_yaml , create_directories

In [6]:
class ConfigurationManager:
    
    def __init__(
        self, 
        config_file_path = CONFIG_FILE_PATH,
        params_file_path = PARAMS_FILE_PATH,
        schema_file_path = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            data_source=config.data_source,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
        

In [7]:
import os 
import urllib.request as request
import zipfile
from Lung_Cancer.logging import logger

In [8]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig) :
        self.config = config

    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info(f"Downloading data from {self.config.data_source}")
            request.urlretrieve(self.config.data_source, self.config.local_data_file)
            logger.info(f"Downloaded data to {self.config.local_data_file}")
        else:
            logger.info(f"Data already exists at {self.config.local_data_file}")    
            
            
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    # data_ingestion.download_data()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    logger.error(e)

[2024-04-21 16:38:29,074]- Lung_Cancer.logging - INFO - yaml file: config/config.yaml loaded successfully
[2024-04-21 16:38:29,076]- Lung_Cancer.logging - INFO - yaml file: params.yaml loaded successfully
[2024-04-21 16:38:29,078]- Lung_Cancer.logging - INFO - yaml file: schema.yaml loaded successfully
[2024-04-21 16:38:29,079]- Lung_Cancer.logging - INFO - created directory at: artifacts
[2024-04-21 16:38:29,080]- Lung_Cancer.logging - INFO - created directory at: artifacts/data_ingestion
